# Quantitative Stock Selection

In [1]:
import fmp # fmp.py contains all helper functions working with FMP API for financial data
import pandas as pd

## 1. Retrieve Dow Jones stock symbols and financial data 

In [2]:
# Retrieve symbols and financials
dow_tickers = fmp.dow_symbols()
df = fmp.extract_financials(dow_tickers)

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30


## 2. Filtering stocks based on financial ratio criteria

In [3]:
# Conditions below indicate that we want the stocks which have positive
# Revenue Growth, Gross Profit Margin, EPS Growth, ROE, DPS Growth and Dividend Yield

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))
df = df[conditions]
df

,Symbol,Name,Exchange,Sector,Industry,MarketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth
1,WBA,"Walgreens Boots Alliance, Inc.",NASDAQ,Healthcare,Pharmaceutical Retailers,42.739007,0.441947,0.106518,0.202202,0.163854,0.038069,1.881,0.304819
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,250.303955,0.595383,0.514070,0.448202,0.043478,0.006952,1.100,0.716614
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,39.098049,0.481540,0.132920,0.249376,0.280277,0.022079,3.460,0.317382
6,VZ,Verizon Communications Inc.,NYSE,Communication Services,Telecom Services,225.119027,0.089299,0.281949,0.586553,0.102362,0.046179,2.511,0.110776
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,218.672300,0.743302,0.231499,0.555936,0.506024,0.025471,1.373,0.374296
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,431.595094,0.338716,0.270051,0.669963,0.012766,0.025252,4.140,0.348304
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,240.959488,0.247370,0.266846,0.640210,0.048827,0.025646,1.460,0.433121
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2269.534487,1.328849,0.462317,0.689258,0.064902,0.007417,2.240,0.553834
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,358.188122,0.813120,7.047047,0.337181,0.173127,0.019004,6.450,1.387776
27,CVX,Chevron Corporation,NYSE,Energy,Oil & Gas Integrated,187.608613,0.266313,0.026990,0.290348,1.236111,0.054221,5.260,0.254691


In [4]:
df.shape # there are 10 stocks which satisfy the financial ratio criteria

(10, 13)

## 3. Trend and Momentum Analysis

In terms of analyzing historial prices, there are several measures we can choose. We can simply look at the relationship between current price and long term moving averages to see where the stock is located, calculate historical momentum to select positive momentum stocks or select the stocks which have outperformed the benchmark.

### 3-1. Select stocks which have outperformed against S&P 500

Since our goal is to outperform the benchmark index which is S&P 500, we can try to select stocks which have outperformed against the benchmark for alpha generation.

### 3-2. Trend analysis using 200 MA

We can also look through the stocks which prices are above 200 MA.

### 3-3. Momentum analysis using historical 1Y returns

In [5]:
# Compute historical momentum

m12_momentums = []
m24_momentums = []
m36_momentums = []
count = 0
for symbol in df["Symbol"]:
    count += 1
    m12_momentums.append(fmp.calculate_hist_momentum(symbol, 252))
    m24_momentums.append(fmp.calculate_hist_momentum(symbol, 504))
    m36_momentums.append(fmp.calculate_hist_momentum(symbol, 756))

df["m12_momentum"] = m12_momentums
df["m24_momentum"] = m24_momentums
df["m36_momentum"] = m36_momentums

positive_mom_stocks = df[(df['m12_momentum'] > 0) & (df['m24_momentum'] > 0)
                         & (df['m36_momentum'] > 0)]
positive_mom_stocks

,Symbol,Name,Exchange,Sector,Industry,MarketCap(B),Revenue_Growth,ROE,GPMargin,EPS_Growth,DivYield,DPS,DPS_Growth,m12_momentum,m24_momentum,m36_momentum
3,NKE,"NIKE, Inc.",NYSE,Consumer Cyclical,Footwear & Accessories,250.303955,0.595383,0.514070,0.448202,0.043478,0.006952,1.100,0.716614,0.354417,0.789282,0.892866
5,TRV,"The Travelers Companies, Inc.",NYSE,Financial Services,Insurance—Property & Casualty,39.098049,0.481540,0.132920,0.249376,0.280277,0.022079,3.460,0.317382,0.391385,0.069639,0.206174
7,INTC,Intel Corporation,NASDAQ,Technology,Semiconductors,218.672300,0.743302,0.231499,0.555936,0.506024,0.025471,1.373,0.374296,0.089626,0.059722,0.207178
9,JNJ,Johnson & Johnson,NYSE,Healthcare,Drug Manufacturers—General,431.595094,0.338716,0.270051,0.669963,0.012766,0.025252,4.140,0.348304,0.122647,0.266927,0.180480
19,CSCO,"Cisco Systems, Inc.",NASDAQ,Technology,Communication Equipment,240.959488,0.247370,0.266846,0.640210,0.048827,0.025646,1.460,0.433121,0.420114,0.161938,0.216939
20,MSFT,Microsoft Corporation,NASDAQ,Technology,Software—Infrastructure,2269.534487,1.328849,0.462317,0.689258,0.064902,0.007417,2.240,0.553834,0.504214,1.203436,1.721776
21,HD,"The Home Depot, Inc.",NYSE,Consumer Cyclical,Home Improvement Retail,358.188122,0.813120,7.047047,0.337181,0.173127,0.019004,6.450,1.387776,0.201565,0.457306,0.614156


## 4. Calculate stocks' current discount rate from 52W high

We don't want to buy the stocks when they are expensive. We want to wait until they are traded at discount or buy at pullback.

In [6]:
watchlist = positive_mom_stocks[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []
count = 0

for symbol in watchlist["symbol"]:
    count += 1
    print(f"{symbol}: {count}/{len(watchlist['symbol'])}")
    currentPrice = fmp.get_current_price(symbol)
    high = fmp.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high * 100

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["Current Price"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount (%)"] = discounts

watchlist = watchlist.sort_values(by="Discount (%)")
watchlist

KeyError: "None of [Index(['symbol', 'name'], dtype='object')] are in the [columns]"

### Example: Consider buying if discounted more than 15% from 52W high

In [ ]:
buylist = watchlist[watchlist['Discount (%)'] < -15]
buylist

In [ ]:
#!jupyter nbconvert --to script --no-prompt analysis.ipynb